In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/Users/calebchiam/Documents/GitHub/Cornell-Conversational-Analysis-Toolkit'

We can use Hypergraph features for various predictive tasks:

In [3]:
from convokit import PairedPrediction

In [4]:
import convokit
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
os.chdir('..')

In [ ]:
os.chdir('reddit-corpus')

In [19]:
os.listdir()

['reddit-corpus']

In [10]:
corpus = convokit.Corpus(filename=convokit.download('reddit-corpus'))
# corpus = convokit.Corpus(filename='reddit-corpus')

Dataset already exists at /Users/calebchiam/.convokit/downloads/reddit-corpus


In [11]:
hc = convokit.HyperConvo(prefix_len=10, min_thread_len=10, include_root=False)
hc.fit_transform(corpus)

In [22]:
remake_cache = True
if remake_cache:
    with open("hyperconvo_feats.p", "wb") as f:
        hyperconvo_feats = {}
        for convo in corpus.iter_conversations():
            hyperconvo_feats.update(convo.meta["hyperconvo"])
        pickle.dump(hyperconvo_feats, f)
else:
    with open("hyperconvo_feats.p", "rb") as f:
        hyperconvo_feats = pickle.load(f)

In [12]:
threads = corpus.utterance_threads(include_root=False)

In [16]:
# Use only the first 10 comments in each thread
thread_pfxs = corpus.utterance_threads(prefix_len=10, include_root=False)

In [17]:
len(threads)

100000

In [18]:
corpus.print_summary_stats()

Number of Users: 521777
Number of Utterances: 2004262
Number of Conversations: 84979


In [19]:
from collections import defaultdict
thread_roots_by_self_post = defaultdict(list)
for top_level_comment, thread in threads.items():
    rt = thread[next(iter(thread))].root
    thread_roots_by_self_post[rt].append(top_level_comment)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
hyperconvo_feats = hc.retrieve_feats(corpus)

In [21]:
path_stats = hc.retrieve_motif_pathway_stats(corpus)

In [22]:
motif_counts = hc.retrieve_motif_counts(corpus)

In [23]:
threads_motifs = hc.retrieve_motifs(corpus)

In [24]:
latent_motif_count = {thread_id: hc._latent_motif_count(motif_dict, trans=False)[0] for thread_id, motif_dict in threads_motifs.items()}

In [27]:
import random

In [28]:
# first generate positive and negative examples based on task

def generate_pos_neg(task: str, post_to_thread_obj, threads, thread_pfxs):
    pos, neg = [], []
    if task == "comment-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos = [root for root in thread_roots if len(threads[root]) >= 15]
            has_neg = [root for root in thread_roots if len(threads[root]) == 10]
            
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    elif task == "commenter-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for root in thread_roots:
                if len(threads[root]) >= 20:
                    if len(set(c.user.name for c in threads[root].values())) >= \
                        len(set(c.user.name for c in thread_pfxs[root].values())) * 2:
                            has_pos.append(root)
                    else:
                        has_neg.append(root)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    print("- {} positive, {} negative pts for {} task".format(len(pos), len(neg), task))
    
    return pos, neg


In [29]:
pos_comment_growth, neg_comment_growth = generate_pos_neg("comment-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 1827 positive, 1827 negative pts for comment-growth task


In [30]:
pos_commenter_growth, neg_commenter_growth = generate_pos_neg("commenter-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 849 positive, 849 negative pts for commenter-growth task


In [31]:
from copy import deepcopy

In [36]:
hyperconv_motif = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_motif.items():
    feats.update(motif_counts[thread_id])

In [54]:
def get_num_users(thread):
    return len(set(utt.user.name for utt in thread.values()))

In [57]:
thread_to_usercount = dict()
for thread_id in thread_pfxs:
    thread_to_usercount[thread_id] = {"num_users": get_num_users(thread_pfxs[thread_id])}

In [64]:
hyperconv_usercount = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_usercount.items():
    feats.update(thread_to_usercount[thread_id])

In [58]:
motifpaths_usercount = deepcopy(path_stats)

In [67]:
latent_usercount = deepcopy(latent_motif_count)
for thread_id, feats in latent_usercount.items():
    feats.update(thread_to_usercount[thread_id])

In [59]:
for thread_id, feats in motifpaths_usercount.items():
    feats.update(thread_to_usercount[thread_id])

In [37]:
hyperconv_paths = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_paths.items():
    feats.update(path_stats[thread_id])

In [38]:
hyperconv_latent = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_latent.items():
    feats.update(latent_motif_count[thread_id])

In [39]:
hyperconv_motifall = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_motifall.items():
    feats.update(motif_counts[thread_id])
    feats.update(path_stats[thread_id])
    feats.update(latent_motif_count[thread_id])

In [69]:
random.seed(2019)
for task in ["comment-growth", "commenter-growth"]: #, "post-deleted", "user-deleted"
    print("TASK: {}\n".format(task))
    
    pos, neg = generate_pos_neg(task, thread_roots_by_self_post, threads, thread_pfxs)

    pp = PairedPrediction()
    pp.fit_predict(hyperconvo_feats, pos, neg)
    
    for feats, name in [(hyperconvo_feats, "hyperconvo"),
                        (hyperconv_usercount, "hyperconv-usercount"),
                        (latent_motif_count, "latentmotif"),
                        (latent_usercount, "latentmotif-usercount"),
                        (path_stats, "motifpaths"),
                        (motifpaths_usercount, "motifpaths-usercount"),
                        (hyperconv_motif, "hyperconv-motif"),
                        (hyperconv_paths, "hyperconv-paths"),
                        (hyperconv_latent, "hyperconv-latent"),
                        (hyperconv_motifall, "hyperconvo-motifall"),
                        (thread_to_usercount, "usercount")
                       ]:
        print("Feature set: {}".format(name))
        pp.fit_predict(feats, pos, neg, test_size=0.2)
        feat_names = list(feats[next(iter(feats))].keys()) 
        pp.print_extreme_coefs(feat_names, num_features=5)

TASK: comment-growth

- 1827 positive, 1827 negative pts for comment-growth task
Excluded {} data point(s) that contained NaN values.
Test accuracy of 0.5506
Feature set: hyperconvo
Excluded {} data point(s) that contained NaN values.
Test accuracy of 0.5506
TOP 5 FEATURES
prop-nonzero[indegree over C->C mid-thread responses]: 1.245
mean[outdegree over C->c responses]: 0.351
mean[outdegree over C->C responses]: 0.351
mean[indegree over C->C responses]: 0.351
norm.max[outdegree over C->c mid-thread responses]: 0.335

BOTTOM 5 FEATURES
max[indegree over C->C responses]: -0.375
entropy[indegree over C->C mid-thread responses]: -0.376
entropy[indegree over C->C responses]: -0.404
mean-nonzero[indegree over C->C responses]: -0.520
prop-nonzero[indegree over C->C responses]: -1.022

Feature set: hyperconv-usercount
Excluded {} data point(s) that contained NaN values.
Test accuracy of 0.5506
TOP 5 FEATURES
prop-nonzero[indegree over C->C mid-thread responses]: 1.246
mean[outdegree over C->c r

Test accuracy of 0.4940
TOP 5 FEATURES
entropy[outdegree over C->c mid-thread responses]: 0.578
entropy[outdegree over C->C mid-thread responses]: 0.578
norm.max[outdegree over c->c mid-thread responses]: 0.540
norm.2nd-largest[outdegree over c->c mid-thread responses]: 0.534
entropy[indegree over C->C mid-thread responses]: 0.442

BOTTOM 5 FEATURES
2nd-largest / max[indegree over C->c responses]: -0.458
max[indegree over C->C mid-thread responses]: -0.527
2nd-largest / max[indegree over C->C mid-thread responses]: -0.546
prop-multiple[indegree over c->c mid-thread responses]: -0.619
prop-multiple[indegree over C->c mid-thread responses]: -0.619

Feature set: latentmotif
Test accuracy of 0.4824
TOP 5 FEATURES
INCOMING_1TO3_TRIADS: 0.453
SINGLE_EDGE_TRIADS: 0.324
DIRECTED_CYCLE_TRIADS: 0.117
INCOMING_2TO3_TRIADS: 0.077
INCOMING_TRIADS: 0.026

BOTTOM 5 FEATURES
OUTGOING_3TO1_TRIADS: -0.078
TRIRECIPROCAL_TRIADS: -0.098
OUTGOING_TRIADS: -0.155
NO_EDGE_TRIADS: -0.238
DIRECIPROCAL_TRIADS: -0

In [78]:
threads_text = dict()
for thread_id in pos + neg:
    threads_text[thread_id] = {"text": " ".join(utt.text for utt in thread_pfxs[thread_id].values())}

In [81]:
text_df = pd.DataFrame.from_dict(threads_text).T

In [ ]:
# First split the pos + neg pairs in train, test sets

In [101]:
from sklearn.model_selection import train_test_split
pos_neg_train, pos_neg_test = train_test_split(list(zip(pos, neg)), test_size=0.2, random_state=42)

In [104]:
pos_train = [x[0] for x in pos_neg_train]
neg_train = [x[1] for x in pos_neg_train] 
pos_test = [x[0] for x in pos_neg_test]
neg_test = [x[1] for x in pos_neg_test]

In [ ]:
# BOW vectorizer on train dataset

In [107]:
train_text_df = text_df.loc[pos_train + neg_train]

In [108]:
test_text_df = text_df.loc[pos_test + neg_test]

In [273]:
cv = CountVectorizer(min_df=0.15, max_df=0.6)

In [274]:
train_text_arr = cv.fit_transform(train_text_df['text'])

In [275]:
train_text_transform_df = pd.DataFrame(train_text_arr.toarray(), columns=cv.get_feature_names(), index=train_text_df.index)

In [276]:
train_text_transform_df.head()

,actually,after,again,against,agree,already,also,always,am,another,...,world,would,wouldn,wrong,www,yeah,year,years,yes,your
dok2j8c,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
e06rqka,1,0,0,1,0,0,2,2,1,2,...,0,0,1,0,0,1,0,0,0,5
dx05x4c,0,1,0,3,0,0,2,0,0,0,...,0,3,0,0,0,0,0,0,0,0
dr9meeo,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
dwcubcp,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [277]:
test_text_arr = cv.transform(text_test['text'])
test_text_transform_df = pd.DataFrame(test_text_arr.toarray(), columns=cv.get_feature_names(), index=test_text_df.index)   


In [278]:
from pandas import DataFrame
def _generate_paired_X_y(feats: DataFrame, pos_ids, neg_ids):

    X, y = [], []
    flip = True

    excluded = 0
    for idx in range(len(pos_ids)):
        pos_feats = np.array(feats.loc[pos_ids[idx]])
        neg_feats = np.array(feats.loc[neg_ids[idx]])

        if (np.isnan(pos_feats).any() or np.isnan(neg_feats).any()):
            excluded += 1
            continue

        if flip:
            y.append(1)
            diff = pos_feats - neg_feats
        else:
            y.append(0)
            diff = neg_feats - pos_feats

        X.append(diff)
        flip = not flip

    if excluded > 0:
        print("Excluded {} data point(s) that contained NaN values.".format(excluded))

    return np.array(X), np.array(y)

In [279]:
X_train, y_train = _generate_paired_X_y(train_text_transform_df, pos_train, neg_train)

In [280]:
X_test, y_test = _generate_paired_X_y(test_text_transform_df, pos_test, neg_test)

In [281]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])       

In [282]:
X_train.shape

(679, 202)

In [283]:
clf = clf.fit(X_train, y_train)

In [284]:
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("- {}: {:.4f} train, {:.4f} test".format("cumulative_bow", train_acc, test_acc))

- cumulative_bow: 0.7555 train, 0.5118 test


In [270]:
def print_extreme_coefs(clf, feature_names, num_features: int = 5):
    coefs = clf.named_steps['logreg'].coef_[0].tolist()

    assert len(feature_names) == len(coefs)

    feats_coefs = sorted(list(zip(feature_names, coefs)), key=lambda x: x[1], reverse=True)

    print()
    print("TOP {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[:num_features]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[-num_features:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [271]:
print_extreme_coefs(clf, cv.get_feature_names(), num_features=20)


TOP 20 FEATURES
definitely: 0.763
we: 0.719
used: 0.664
stuff: 0.661
work: 0.649
say: 0.649
remember: 0.641
feel: 0.631
else: 0.629
she: 0.613
some: 0.590
last: 0.584
wrong: 0.576
found: 0.573
ever: 0.571
here: 0.547
thought: 0.507
who: 0.502
believe: 0.465
really: 0.463

BOTTOM 20 FEATURES
had: -0.453
before: -0.466
own: -0.479
literally: -0.489
op: -0.497
use: -0.515
either: -0.519
www: -0.520
great: -0.525
person: -0.536
maybe: -0.541
exactly: -0.541
didn: -0.552
gt: -0.562
someone: -0.569
by: -0.635
people: -0.662
big: -0.685
life: -0.725
each: -1.090



In [ ]:
clf = LogisticRegression(solver="liblinear")
clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("- {}: {:.4f} train, {:.4f} test".format(name, train_acc, test_acc))

In [ ]:
import random
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split

random.seed(2019)

for task in ["comment-growth", "commenter-growth"]: #, "post-deleted", "user-deleted"
    print("TASK: {}\n".format(task))
    
    pos, neg = generate_pos_neg(task, thread_roots_by_self_post, threads, thread_pfxs)

    X, y = generate_paired_features(hyperconvo_feats, pos, neg)
    X_motifcnt, y_motifcnt = generate_paired_features(motif_counts, pos, neg)
    X_latent, y_latent = generate_paired_features(latent_motif_count, pos, neg)
    X_path, y_path = generate_paired_features(path_stats, pos, neg)
    X_hcmotif, y_hcmotif = generate_paired_features(hyperconv_motif, pos, neg)
    X_hcpath, y_hcpath = generate_paired_features(hyperconv_paths, pos, neg)
    X_hclatent, y_hclatent = generate_paired_features(hyperconv_latent, pos, neg)
    X_all, y_all = generate_paired_features(hyperconv_motifall, pos, neg)
    for X, y, feats, name in [(X, y, hyperconvo_feats, "hyperconv"),
                       (X_motifcnt, y_motifcnt, motif_counts, "motifcount"),
                       (X_latent, y_latent, latent_motif_count, "latentmotif"),
                       (X_path, y_path, path_stats, "motifpaths"),
                       (X_hcmotif, y_hcmotif, hyperconv_motif, "hyperconv-motif"),
                       (X_hcpath, y_hcpath, hyperconv_paths, "hyperconv-paths"),
                       (X_hclatent, y_hclatent, hyperconv_latent, "hyperconv-latent"),
                       (X_all, y_all, hyperconv_motifall, "hyperconvo-motifall")
                      ]:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])
        clf.fit(X_train, y_train)

        train_acc = clf.score(X_train, y_train)
        test_acc = clf.score(X_test, y_test)
        print("- {}: {:.4f} train, {:.4f} test".format(name, train_acc, test_acc))
        print_extreme_coefs(clf, feats)